In [62]:
import pandas as pd
import math
from numpy import log as ln
from bokeh.plotting import figure, output_notebook, reset_output, show, save, ColumnDataSource, output_file
from bokeh.models.tools import HoverTool
from bokeh.models import Label
from bokeh.layouts import gridplot, row
from bokeh.transform import factor_cmap, linear_cmap
from bokeh.palettes import inferno, Viridis256, RdYlGn
from bokeh.embed import components
from bokeh.tile_providers import get_provider, Vendors

In [11]:
df = pd.read_excel('corr.xlsx')
df = df.drop_duplicates()
df.head(5)

,Type,Lenth,Width,Depth,ImgN,CS_ID
0,Коррозия,110,150,0.6,1,1
1,Коррозия,700,1050,0.7,2,2
2,Коррозия,1660,700,1.1,3,3
3,Коррозия,320,110,0.6,4,1
4,Коррозия,2150,730,0.7,5,2


Для примера использован датасет всего с двумя типами дефектов (Type): коррозия и КРН; их геометрическими параметрами (Lenth, Width, Depth); номером файла с картинкой (ImgN): например, "1.jpg"; и идентификатором объекта, на котором эти дефекты были обнаружены (CS_ID).

In [12]:
#параметры трубы по умолчанию
t=16
P = 5.5
Dn = 1020
s = 588

#расчет ранга опасности
def Rang(c):
    if c['Type'] == "Коррозия":
        a = P * (Dn - t) / (2 * t * s)
        x = (c['Lenth'] / math.sqrt(Dn * t)) ** 2
        Q = math.sqrt(1 + 0.31 * x)
        E = c['Depth'] / t 
        Ep = (a - 1) * Q / (a - Q)
        return min(E / Ep, 1)

    elif c['Type'] == "КРН":
        if c['Depth'] >= 4.0:
            return 1.0
        elif c['Depth'] >= 2.5:
            return 0.8
        else:
            return 0.3
    else:
        return 0

In [13]:
df['Rang'] = df.apply(Rang, axis=1)
df.head(5)

,Type,Lenth,Width,Depth,ImgN,CS_ID,Rang
0,Коррозия,110,150,0.6,1,1,0.039031
1,Коррозия,700,1050,0.7,2,2,0.056262
2,Коррозия,1660,700,1.1,3,3,0.093397
3,Коррозия,320,110,0.6,4,1,0.044000
4,Коррозия,2150,730,0.7,5,2,0.059994


In [14]:
#нам нужна только пловина палитры inferno, поэтому для получения 100 цветов для отображения рангов разбиваем палитру на 200
colors200 = inferno(200)
def appendcolor(c):
    colind = round(c['Rang'] * 100)
    return colors200[colind]
df['Color'] = df.apply(appendcolor, axis=1)

In [15]:
#расчитаем радиус круга такой, чтобы его площадь бала равна площади отображаемого дефекта
def rad_by_area(c):
    return math.sqrt(c['Width']*c['Lenth'] / math.pi)
df['Radius'] = df.apply(rad_by_area, axis=1)

In [63]:
# используем визуализацияю scatter, чтобы продемонстрировать, какие из дефектов наиболее опасны (спойлер: не обязательно самые большие)
source = ColumnDataSource(df)
TOOLS="crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,"
pl = figure(
    plot_width=600,
    plot_height=600,
    title='Распределение дефектов по рангам опасности',
    x_axis_label='Длина, мм',
    y_axis_label='Глубина, мм',
    tools=TOOLS,
    sizing_mode = 'fixed'
)

hover = HoverTool()
hover.tooltips = """
  <div>
    <h3>@Type</h3>
    <div><strong>Глубина: </strong>@Depth</div>
    <div><strong>Ранг: </strong>@Rang</div>
    <div><img src="Corrosion_Images/@ImgN.jpg" alt="" width="200" /></div> 
  </div>
"""
pl.add_tools(hover)

pl.scatter('Lenth', 'Depth', radius='Radius',
          fill_color='Color', fill_alpha=0.6,
          line_color=None,
          source=source)

citation = Label(x=10, y=10, x_units='screen', y_units='screen',
                 text='Площадь кругов соотвествует площади дефектов', render_mode='css',
                 border_line_color='black', border_line_alpha=1.0,
                 background_fill_color='white', background_fill_alpha=1.0)
pl.add_layout(citation)

try:
    reset_output()
    output_notebook()
    show(pl)  
except:
    output_notebook()
    show(pl)  

Loading BokehJS ...

Для оформления легенды в Bokeh предусмотрена модель ColorBar, однако для вывода в Notebook она не подходит. Пойдем другим путем: в роли легенды будет отдельная диаграмма.

In [52]:
po = figure(
    plot_width=100,
    plot_height=600,
    title='Ранг',
    tools = '',
    sizing_mode = 'fixed'
)

colors20 = inferno(20) #тот же прием: отрезаем от палитры половину
po.rect(x=0, y=[0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], width=0.1, height=0.1,
       fill_color=colors20[:11],
       line_color=None)
po.xaxis.visible = False

try:
    reset_output()
    output_notebook()
    show(po)  
except:
    output_notebook()
    show(po)

Loading BokehJS ...

## Работа с геоданными 

In [26]:
df_geo = pd.read_excel('SampleGeo.xlsx')
df_geo = df_geo.drop_duplicates()
df_geo['Radius'] = df_geo['Pts'] *100
df_geo.head(5)

,CS_ID,CS_Name,lon,lat,Pts,Omega,Radius
0,1,Компрессорная станция «Ржевская»,34.199055,56.286839,0.24,0.90,24.0
1,2,Компрессорная станция «Елизаветинская»,30.200890,60.437593,0.05,0.56,5.0
2,3,Компрессорная станция «Новоюбилейная»,41.989120,59.660765,0.32,0.00,32.0


In [ ]:
Три условных объекта, к которым мы отнесли наши дефекты, характеризуются географической широтой и долготой и показателями технического состояния Птс и Омега (их расчет оставим за скобками).
Оба показателя меньше 1, поэтому для удобства отображения кругов на карте зададим их радиус, умножив Птс на 100.

Bokeh работает с шеошрафическими координатами в проекции Меркатора, поэтому широту и долготу необходимо преобразовать для корректного отображения:

In [27]:
def merc_x(geo):
    k = 6378137
    return geo['lon'] * (k * math.pi/180.0)
def merc_y(geo):
    k = 6378137
    return ln(math.tan((90 + geo['lat']) * math.pi/360.0)) * k

In [28]:
df_geo['merc_x'] = df_geo.apply(merc_x, axis=1)
df_geo['merc_y'] = df_geo.apply(merc_y, axis=1)
df_geo

,CS_ID,CS_Name,lon,lat,Pts,Omega,Radius,merc_x,merc_y
0,1,Компрессорная станция «Ржевская»,34.199055,56.286839,0.24,0.90,24.0,3.807021e+06,7.615730e+06
1,2,Компрессорная станция «Елизаветинская»,30.200890,60.437593,0.05,0.56,5.0,3.361948e+06,8.497814e+06
2,3,Компрессорная станция «Новоюбилейная»,41.989120,59.660765,0.32,0.00,32.0,4.674207e+06,8.324595e+06


Теперь можно построить визуализацию на карте:

In [55]:
tile_provider = get_provider(Vendors.CARTODBPOSITRON)
geo_source = ColumnDataSource(df_geo)


pm = figure(x_range=(3000000, 5000000), y_range=(7000000, 9000000),
           x_axis_type="mercator", y_axis_type="mercator",
            title='Распределение объектов по показателям технического состояния',
            plot_width=600,
            plot_height=600,
           tools=TOOLS, 
            sizing_mode = 'fixed')
pm.add_tile(tile_provider)


pm.circle(x="merc_x", y="merc_y", size='Radius', 
          fill_color= linear_cmap('Omega', RdYlGn[11], low=1, high=0), 
          fill_alpha=0.8, 
          line_color='black',
          source=geo_source)


hover_geo = HoverTool()
hover_geo.tooltips = """
  <div>
    <h3>@CS_Name</h3>
    <div><strong>Птс: </strong>@Pts</div>
    <div><strong>Омега: </strong>@Omega</div>
  </div>
"""
pm.add_tools(hover_geo)

citation = Label(x=10, y=10, x_units='screen', y_units='screen',
                 text='Площадь кругов соотвествует величине Птс', render_mode='css',
                 border_line_color='black', border_line_alpha=1.0,
                 background_fill_color='white', background_fill_alpha=1.0)
pm.add_layout(citation)


try:
    reset_output()
    output_notebook()
    show(pm)  
except:
    output_notebook()
    show(pm)  

Loading BokehJS ...

In [ ]:
И легенду к ней:

In [53]:
pn = figure(
    plot_width=100,
    plot_height=600,
    title='Омега',
    tools = '',
    sizing_mode = 'fixed'
)
RdYlGn[11].reverse() #чтобы большие значения Омеги отображались "хорошим" зеленым цветом, "перевернем" палитру
pn.rect(x=0, y=[0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], width=0.1, height=0.1,
       fill_color=RdYlGn[11],
       line_color=None)
pn.xaxis.visible = False

try:
    reset_output()
    output_notebook()
    show(pn)  
except:
    output_notebook()
    show(pn)

Loading BokehJS ...

А теперь выведем результаты в один ряд:

In [59]:
grid = row(pm, pn, pl, po)
try:
    reset_output()
    output_notebook()
    show(grid)  
except:
    output_notebook()
    show(grid)



Loading BokehJS ...

Loading BokehJS ...

ToDo: связать датасеты через иденттификатор объета и настроить показ только тех дефектов (правая панель), которые соответствуют выбранному объекту (левая панель)